# **Word2Vec 분석하기**
1. **Frequency 분석** ( **문서의 개요** 살펴보기)
1. **Tf IDF** 분석 ( **특정 모집단 내 일부 문서의 특징** 찾기)
1. **Word2Vec 모델의** 분석 ( **적은 데이터로 키워드** 특징찾기)

## **1 PreProcessing 2 유효문장 선별하기**
**Word2vec** 을 만들기 위해서는 **문장 데이터로** 전처리가 필요하다
1. **그냥 만든경우** 와, **문장을 선별한 경우를** 비교하여 성능 향상 시키기
1. **핵심 Token 을 찾고**, 이를 바탕으로 word2vec 결과값 필터링 하기 **(Core 찾기가 우선)**
1. 여기선 **문장내 단어가 3개 이상 출연한** 문장만 선별하기

```
'NNB': '의존 명사',
'NNBC': '단위를 나타내는 명사',
'NNG': '일반 명사',
'NNP': '고유 명사',
'NP': '대명사',
'VA': '형용사',
'VV': '동사',
'XSA': '형용사 파생 접미사',
'XSN': '명사파생 접미사',
'XSV': '동사 파생 접미사'
```

In [2]:
from tqdm import tqdm
import pandas as pd
df        = pd.read_csv("data/naverPost_muyong.csv")
df.head(3)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림\n부드러운 닭가슴살 장조림 만들기\n베리츄\n\n2017.9\n1....
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료\n수비드 닭가슴살 요리 돈까스 볶음밥 만들기\n주니호맘\n\n20...
2,가슴살_닭_라이스_카레,food\n닭의품격 초간편 냉동닭가슴살 카레라이스 만들기\n유누네\n\n2018.9...


In [3]:
key_posts = df.Posts.tolist()
result    = []

for posts in tqdm(key_posts[:3]):
    posts_temp = []
    for post in posts.split("|"): # 500 개 문장
        for _ in post.split("\n"):
            if len(_.split(" ")) >= 3: # 3단어 이상으로 구성된 문장만
                posts_temp.append(_.strip())
    result.append("\n".join(posts_temp))
# print(result[0][:50])
# df.head(2)

100%|██████████| 3/3 [00:00<00:00, 176.57it/s]

반찬 볶음 조림
부드러운 닭가슴살 장조림 만들기
본문 기타 기능
조림 밑반찬으로 대표적인 


,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림\n부드러운 닭가슴살 장조림 만들기\n베리츄\n\n2017.9\n1....
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료\n수비드 닭가슴살 요리 돈까스 볶음밥 만들기\n주니호맘\n\n20...


In [4]:
# Posts 0번 키워드가 본문에 몇번 출현했는지 확인
post_temp = []
for _ in df.Posts[0].splitlines():
    post_temp.extend(_.split(" "))

# Query 내용 살펴보기
from collections import Counter
for _ in df.Query[0].split("_"):
    print("Key Token: {:4}  Count: {}".format(_.strip(), Counter(post_temp)[_]))

Key Token: 가슴살   Count: 17
Key Token: 계란    Count: 110
Key Token: 닭     Count: 33
Key Token: 장조림   Count: 377


In [13]:
Okt().pos("짜장면 부대찌개 탕수육 맛있게 시원하게 짭조름하다", stem=True)

[('짜장면', 'Noun'),
 ('부대찌개', 'Noun'),
 ('탕수육', 'Noun'),
 ('맛있다', 'Adjective'),
 ('시원하다', 'Adjective'),
 ('짜다', 'Verb'),
 ('조르다', 'Verb')]

In [14]:
Mecab().pos("짜장면 부대찌개 탕수육 맛있게 시원하게 짭조름하다")

[('짜장면', 'NNG'),
 ('부대찌개', 'NNG'),
 ('탕수육', 'NNG'),
 ('맛있', 'VA'),
 ('게', 'EC'),
 ('시원', 'XR'),
 ('하', 'XSA'),
 ('게', 'EC'),
 ('짭조름', 'XR'),
 ('하', 'XSA'),
 ('다', 'EC')]

In [35]:
" // ".join(list(Mecab().tagset.keys()))

'EC // EF // EP // ETM // ETN // IC // JC // JKB // JKC // JKG // JKO // JKQ // JKS // JKV // JX // MAG // MAJ // MM // NNB // NNBC // NNG // NNP // NP // NR // SC // SE // SF // SH // SL // SN // SSC // SSO // SY // VA // VCN // VCP // VV // VX // XPN // XR // XSA // XSN // XSV'

In [36]:
" // ".join(list(Okt().tagset.keys()))

'Adjective // Adverb // Alpha // Conjunction // Determiner // Eomi // Exclamation // Foreign // Hashtag // Josa // KoreanParticle // Noun // Number // PreEomi // Punctuation // ScreenName // Suffix // Unknown // Verb'

In [38]:
from konlpy.tag import Mecab, Okt
df.Posts[0][:100]
tokens = [_[0]  for _ in Okt().pos(df.Posts[0], stem=True)
                if _[1] in ['Adverb','Verb','Noun','Adjective']]
"/ ".join(sorted(list(set(tokens))))[:600]

'가감/ 가게/ 가격/ 가공/ 가기/ 가까이/ 가깝다/ 가꾸다/ 가끔/ 가능/ 가능하다/ 가다/ 가도/ 가두다/ 가득/ 가득하다/ 가랑이/ 가량/ 가로/ 가루/ 가르다/ 가리비/ 가만히/ 가면/ 가미/ 가버리다/ 가벼워지다/ 가볍다/ 가성/ 가세/ 가스/ 가스레인지/ 가슴/ 가시다/ 가쓰오부시/ 가씨/ 가야/ 가열/ 가오/ 가요/ 가운데/ 가위/ 가을/ 가입/ 가장/ 가장자리/ 가정/ 가정식/ 가족/ 가족사진/ 가지/ 가지다/ 가지런하다/ 각/ 각각/ 각종/ 간/ 간간/ 간간히/ 간과/ 간다/ 간단/ 간단하다/ 간도/ 간만/ 간식/ 간의/ 간이/ 간장/ 간장게장/ 간편하다/ 간혹/ 갈고/ 갈기갈기/ 갈다/ 갈라/ 갈릭/ 갈비/ 갈색/ 갈아/ 갈치/ 감/ 감기/ 감다/ 감사하다/ 감싸다/ 감안/ 감염/ 감자/ 감칠맛/ 감탄/ 갑/ 갑자기/ 값/ 갓/ 강/ 강된장/ 강렬하다/ 강불/ 강원/ 강중/ 강추/ 강하다/ 강화유리/ 갖다/ 갖은/ 갖추다/ 같다/ 같이/ 개/ 개그맨/ 개다/ 개도/ 개똥/ 개르/ 개봉/ 개운하다/ 개월/ 개월땐/ 개인/ 개정/ 개취/ 개폐/ 갯수/ 걍/ 거/ 거기/ 거나/ 거들다/ 거랍니/ 거르기/ 거른/ 거름/ 거리/ 거릴정도/ 거무틱틱/'

In [53]:
from konlpy.tag import Mecab, Okt
df.Posts[0][:100]
tokens = [_[0]  for _ in Mecab().pos(df.Posts[0]) 
       if _[1] in ['XR','VA','VV']]
" // ".join(list(set(tokens)))[:300]

'풀어놓 // 날아가 // 지키 // 절묘 // 아프 // 뜨거우 // 캐 // 무섭 // 유명 // 진하 // 쓰이 // 근사 // 부대끼 // 무 // 은은 // 후덥지근 // 돌려먹 // 높 // 꼼꼼 // 나가 // 말캉 // 오르 // 만드 // 한가 // 넘어가 // 연하 // 잽싸 // 꺼림 // 묻 // 삶 // 짭조름 // 이렇 // 못지않 // 기 // 차갑 // 녹 // 채우 // 찢어지 // 먹히 // 퍼지 // 기다리 // 부드럽 // 하얗 // 들 // 걷어가 // 배어드 // 배 // 잠기 // 따 // 눌'

In [55]:
# 단어의 전처리 작업 진행하기
# ['NNB','NNBC','NNG','NNP','NP','VA','VV','XR','XSA','XSN','XSV']
# 
from konlpy.tag import Mecab, Okt
df.Posts[0][:100]
tokens = [_[0]  for _ in Mecab().pos(df.Posts[0]) 
       if _[1] in ['XR','VA','VV']]

In [57]:
Mecab().tagset.keys()

dict_keys(['EC', 'EF', 'EP', 'ETM', 'ETN', 'IC', 'JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX', 'MAG', 'MAJ', 'MM', 'NNB', 'NNBC', 'NNG', 'NNP', 'NP', 'NR', 'SC', 'SE', 'SF', 'SH', 'SL', 'SN', 'SSC', 'SSO', 'SY', 'VA', 'VCN', 'VCP', 'VV', 'VX', 'XPN', 'XR', 'XSA', 'XSN', 'XSV'])

## **2 Word2Vec from String**
**[Gensim Tutorial](https://radimrehurek.com/gensim/models/word2vec.html)**, ||| **[w2v from Sentence block](https://codesachin.wordpress.com/2015/10/09/generating-a-word2vec-model-from-a-block-of-text-using-gensim-python/)**  
1. fasttext 및 gensim 에서는 다음과 같이 파일로 저장한 뒤 활용을 합니다.
1. 임시파일을 계속 만들고 저장하는 작업을 줄이기 위해선 별도의 접근방법이 필요합니다.

```python
%%time
from gensim.models import word2vec
txt_file = 'w2v_script.txt'
data  = word2vec.LineSentence(txt_file)
model = word2vec.Word2Vec(data, size=30, window=2, min_count=16, 
                          hs=1, workers=4, iter=100, sg=1)
model.save(model_file)
```

```python
>>> from gensim.models import Word2Vec
>>> sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
>>>
>>> model = Word2Vec(min_count=1)
>>> model.build_vocab(sentences)  # prepare the model vocabulary
>>> model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # train word vectors
(1, 30)
```

In [60]:
# df.Posts[0].splitlines()[:5]
# result[0].splitlines()[:3]
# w2v_sentences = [_.split(" ") for _ in result[0].splitlines()]

w2v_sentences = []
for _ in df.Posts[0].splitlines():
    tokens = Okt().pos(_, stem=True)
    tokens = [t[0] for t in tokens  if t[1] in ['Adverb','Verb','Noun','Adjective']]
    if len(tokens) > 3:  # 갯수가 3개 이상있는 자료만 추출
        w2v_sentences.append(tokens)
        
# result[0].splitlines()[:3]
# w2v_sentences = [_.split(" ") for _ in result[0].splitlines()]

In [91]:
# defined Model
from gensim.models import Word2Vec
w2v_model  = Word2Vec(sg=1, hs=1, workers=4, size=100, min_count=5, window=2)
w2v_string = w2v_model.build_vocab(w2v_sentences)
w2v_model.train(w2v_sentences, total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

/home/momukji/Python/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(106980, 171340)

In [92]:
" ".join(sorted(w2v_model.wv.vocab.keys()))

'가감 가격 가끔 가능 가능하다 가다 가득 가량 가루 가슴 가운데 가장 가족 가지 간 간간 간단 간단하다 간만 간이 간장 간편하다 갈다 갈릭 갈색 감기 감다 감자 감칠맛 강하다 같다 같이 개 개월 개인 개정 걍 거 거기 거나 거들다 거의 거품 걱정 건 건강 건강하다 건지다 걷다 걸 걸다 걸리다 것 겉 게 겨울 결 경우 계란 계란말이 계량 계속 계시다 고기 고르다 고민 고소하다 고추 고추가 고추장 곤약 골고루 곳 공감 공기 과 과정 괜찮다 구 구매 구멍 구쁘 구입 국 국물 국자 굳다 굽 굽다 귀찮다 그 그것 그냥 그대로 그동안 그래도 그래서 그렇게 그렇다 그릇 근데 글 금방 급하다 기다 기도 기름 기름기 기분 기억 기한 기호 김 김치 김치볶음밥 깍 깔끔하다 깔다 깨 깨끗이 깨끗하다 깨다 깨지다 꺼내다 껍질 꼬동 꼭 꼭지 꽈리 꽤 꿀 끄다 끓다 끓이다 끝 끼 나 나가다 나다 나머지 나물 나서다 나오다 나중 난 날 날씨 남기다 남다 남편 낮추다 낳다 내 내다 내용 내일 내주다 냄비 냄새 냉동 냉장 냉장고 너무 넉넉하다 넘다 넘치다 넣기 넣다 네이버 노른자 놀다 높다 놓다 누르다 눈 느끼다 느낌 늘 늘다 다 다가 다르다 다른 다시 다시마 다양하다 다음 다이어트 다지다 다해 단 단단하다 단맛 단백질 단호박 닫다 달 달걀 달다 달달 닭 닭고기 담 담그다 담다 담백하다 당근 대다 대략 대신 대체 대충 대파 대하 대한 댓글 더 더욱 더하다 덜 덜다 덥다 덩어리 덩이 덮다 데 데친 도 도둑 도록 도시락 도전 돌다 돌리다 동생 동안 동원 돼다 돼지고기 되다 되어다 두다 두르다 두부 뒤 드디어 드리다 든든하다 들다 들어가다 들어주다 듬뿍 듯 등 등등 따다 따뜻하다 따라서 따로 따르다 딱 때 때문 땐 땡초 떨어지다 떼다 또 또는 또한 뚜껑 뚝딱 뜨겁다 뜨다 뜯다 라면 라이스 란 랍니 래야 레시피 로 로그아웃 로만 를 리 리릭 리뷰 마늘 마르다 마무리 마약 마음 마지막 마트 막 만 만나다 만두 만들기 만들다 만들어지다 만만하다 만큼 만하 많다 많이 말 말다 말리다 맘 맛 맛술 맛있다

In [93]:
w2v_model.wv.most_similar(["가슴","살", "계란", "닭", "장조림"], topn=20)

[('살다', 0.9665916562080383),
 ('만들기', 0.9546926021575928),
 ('요리', 0.9442335963249207),
 ('통조림', 0.9365988969802856),
 ('밑반찬', 0.9272643327713013),
 ('닭고기', 0.9268741607666016),
 ('이용', 0.9266475439071655),
 ('살로', 0.9259218573570251),
 ('만들다', 0.9172545075416565),
 ('법', 0.9092066287994385),
 ('살이', 0.9082906246185303),
 ('완성', 0.9069112539291382),
 ('동원', 0.8983728885650635),
 ('다음', 0.8975261449813843),
 ('달걀', 0.886329174041748),
 ('팩', 0.8819456696510315),
 ('굽', 0.8804188966751099),
 ('수비드', 0.8714650869369507),
 ('찢다', 0.8665907979011536),
 ('훈제', 0.8614259362220764)]

In [101]:
test.python

NameError: name 'test' is not defined

In [100]:
w2v_model.wv.most_similar(positive=["단맛"], negative=["싱겁다"], topn=30)

[('앱', 0.4227365553379059),
 ('업데이트', 0.39159780740737915),
 ('최신', 0.38090571761131287),
 ('알림', 0.3807673454284668),
 ('버전', 0.3771092891693115),
 ('올리고당', 0.3755277097225189),
 ('거나', 0.3750928044319153),
 ('설탕', 0.37458086013793945),
 ('술', 0.36780059337615967),
 ('전송', 0.3664911091327667),
 ('네이버', 0.362702876329422),
 ('물엿', 0.331267774105072),
 ('월계수', 0.324979305267334),
 ('컵', 0.3203204870223999),
 ('아가', 0.30791905522346497),
 ('청주', 0.3052445650100708),
 ('할인', 0.30425143241882324),
 ('스푼', 0.30377405881881714),
 ('작성', 0.30159181356430054),
 ('공감', 0.2998657822608948),
 ('맛술', 0.2790175974369049),
 ('잎', 0.2788892686367035),
 ('식감', 0.2672884166240692),
 ('통후추', 0.26368245482444763),
 ('매실', 0.26293912529945374),
 ('살캔', 0.26154229044914246),
 ('얼마', 0.25505203008651733),
 ('보내다', 0.2521663010120392),
 ('반', 0.24980899691581726),
 ('꿀', 0.24892687797546387)]

## **3 유의미한 단어 찾기**

In [ ]:
word2vec_ = thrones2vec = w2v.Word2Vec(
    sg = 1, seed = seed,
    workers = num_workers,
    size = num_features,
    min_count = min_word_count,
    window = context_size,
    sample = downsampling
)

In [49]:
%%time
from gensim.models import word2vec
data = Phrases(sentences=result[0])
# data  = word2vec.LineSentence(script_file)
# model = word2vec.Word2Vec(data, size=30, window=2, min_count=16, hs=1,
#                           workers=4, iter=100, sg=1)
# model.save(model_file)

CPU times: user 151 ms, sys: 0 ns, total: 151 ms
Wall time: 149 ms


In [50]:
model = word2vec.Word2Vec(data, size=30, window=2, min_count=16, 
                          workers=4, iter=100, sg=1)

TypeError: 'int' object is not iterable

In [14]:
type(sentence)

gensim.models.phrases.Phrases

In [6]:
# Post 문장 내용 확인하기
w2v_temp    = result[0].splitlines() # "\n" 기준으로 나눈다
result_temp = []
for _ in w2v_temp:
    result_temp += _.split(" ")

가슴살 16
계란 104
닭 31
장조림 342


In [9]:
"11".isdigit()

True

In [11]:
# 명사 Token 작업된 자료를 ssResport.txt 로 저장 
script_file = './w2v_script.txt'
with open(script_file, 'w', encoding='utf-8') as file:
    file.write(result[0])
    
print("전처리 블로그 내용의 저장 : {}".format(script_file))

전처리 블로그 내용의 저장 : ./w2v_script.txt


## **1 Sentenced Text**
**Word2vec** 을 만들기 위해서는 **문장 데이터로** 전처리가 필요하다
1. **그냥 만든경우** 와, **문장을 선별한 경우를** 비교하여 성능 향상 시키기
1. **핵심 Token 을 찾고**, 이를 바탕으로 word2vec 결과값 필터링 하기 **(Core 찾기가 우선)**

In [36]:
%%time
from gensim.models import word2vec
data  = word2vec.LineSentence(script_file)
model = word2vec.Word2Vec(data, size=30, window=2, min_count=16, hs=1,
                          workers=4, iter=100, sg=1)
model.save(model_file)

Word 2 Vec 모델 파일 ./w2v_recipe.model 저장완료
CPU times: user 4.63 s, sys: 103 ms, total: 4.73 s
Wall time: 3.29 s


In [38]:
model.wv.most_similar(["가슴살", "계란", "닭", "장조림"], topn=20)

[('닭가슴살', 0.725135326385498),
 ('이용한', 0.6401489973068237),
 ('메추리알', 0.6393377780914307),
 ('요리', 0.6060360670089722),
 ('오늘', 0.5656761527061462),
 ('법', 0.5500603318214417),
 ('오늘은', 0.5277817845344543),
 ('닭가슴살과', 0.4969708025455475),
 ('장조림을', 0.4929303228855133),
 ('글입니다', 0.4797065854072571),
 ('만드는', 0.4728950560092926),
 ('만들기', 0.46609869599342346),
 ('위해', 0.4544617533683777),
 ('먹는', 0.44983047246932983),
 ('좋아요', 0.4398360848426819),
 ('꽈리고추', 0.43841955065727234),
 ('다이어트', 0.4327726662158966),
 ('닭고기', 0.4298769235610962),
 ('마늘', 0.42846792936325073),
 ('닭가슴살을', 0.42231449484825134)]

In [41]:
model.wv.most_similar(["가슴살", "계란", "닭", "장조림"], 
                      negative=['닭가슴살', '닭가슴살과', '닭가슴살을'],
                      topn=30)

[('요리', 0.6024991273880005),
 ('사용', 0.3841325342655182),
 ('법', 0.3550140857696533),
 ('글입니다', 0.35304853320121765),
 ('꽈리고추', 0.35070517659187317),
 ('이제', 0.3504452109336853),
 ('간단하게', 0.3490484952926636),
 ('위해', 0.3488353192806244),
 ('먹기', 0.3451535701751709),
 ('오늘', 0.34397000074386597),
 ('다이어트', 0.3195529878139496),
 ('메추리알', 0.31017300486564636),
 ('마늘', 0.3066818416118622),
 ('반찬으로', 0.2982470989227295),
 ('맛있어요', 0.2916230857372284),
 ('레시피', 0.28354838490486145),
 ('오늘은', 0.271423876285553),
 ('내용', 0.26687198877334595),
 ('장조림을', 0.26355087757110596),
 ('좋아요', 0.25986653566360474),
 ('글에', 0.2506791651248932),
 ('넣어서', 0.23929870128631592),
 ('만들', 0.23346063494682312),
 ('이용한', 0.23234985768795013),
 ('안', 0.2303457111120224),
 ('저작자', 0.22936281561851501),
 ('좋아하는', 0.22905974090099335),
 ('물에', 0.22215133905410767),
 ('자주', 0.21964050829410553),
 ('하나', 0.20812654495239258)]

In [34]:
import pandas as pd
posts = pd.read_csv("data/naverPost_muyong.csv")
posts.head(2)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,가슴살_굴소스_닭_볶음밥,아임닭 7기 종료.수비드 닭가슴살 요리 돈까스 볶음밥 만들기.주니호맘..2019.4...


In [4]:
post_temp = posts.loc[0]
post_temp_query = post_temp.Query.split("_")
post_temp_list = post_temp.Posts.split('|')

In [5]:
post_temp_query, post_temp_list[0][:300]

(['가슴살', '계란', '닭', '장조림'],
 '반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32.URL 복사.이웃추가.본문 기타 기능.번역보기.조림 밑반찬으로 대표적인 장조림.지방이 적고 단백질은 풍부한.닭가슴살.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.거기에.새송이버섯.과 제철을 맞이한.꽈리고추.까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살장조림.닭가슴살 장조림 재료.조리시간 1시간 이내 ')

In [6]:
# post 내용의 필터링
post_temp_list_blog = post_temp_list[0]
post_temp_list_blog = post_temp_list_blog.split('.')

In [7]:
import re

In [11]:
# 우선은 간단하게 기능을 추가한뒤
# 식재료 Stemming 보완이 필요한 듯 ex) 후춧가루, 후추, 양념장 과 같은...
#
".".join([sent for sent in post_temp_list_blog   if len(re.findall(r"\w+", sent)) >= 3])

'반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.본문 기타 기능.조림 밑반찬으로 대표적인 장조림.지방이 적고 단백질은 풍부한.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.과 제철을 맞이한.까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살 장조림 재료.조리시간 1시간 이내 난이도 중.생 닭가슴살 200g 물 1L 5컵 청주 2큰술 마늘 4쪽 생강 마늘 크기 1톨.새송이버섯 2개 꽈리고추 200g 약 3줌.양념 고기 육수 300ml 1.5컵 양조간장 100ml 0.5컵 꿀 2큰술 표고버섯가루 1 2큰술 후춧가루 약간.괄호 속 기재된 컵 은 종이컵 기준입니다.꿀 대신 올리고당으로 대체 가능합니다.고기 삶은 물은 양념에 사용할 거예요.끓는 물에 닭 가슴살 물 1L 5컵 마늘 4쪽 생강 1톨 청주 2큰술을 넣고 삶아주세요.센 불에서 끓어오르면 중불에서 뚜껑을 덮어 20 25분 정도 삶으면 됩니다.삶은 물은 버리지 마세요.삶는 시간이 생각보다 오래 걸리기 때문에 이때 재료 손질 및 양념을 만들어두는 것이 좋아요.닭이 충분히 익었는지 확인할 땐 젓가락을 이용해 찔러보세요.젓가락이 푹 들어가고 핏물이 빠져나오지 않으면 충분하게 익은 거예요.고기 삶는 동안 꽈리고추를 흐르는 물에 깨끗하게 씻은 뒤 꼭지를 제거해주세요.포크를 이용해 구멍을 내주세요.양념이 잘 배도록 구멍을 내주는 것인데 작은 건 한번 긴 건 두 번 정도 콕 찔러주면 됩니다.길이가 너무 긴 것은 반으로 썰어주세요.꽈리고추 고르는 법.전체적으로 윤기가 나며 꼭지가 단단한 것을 고르면 되는데요.이때 몸통이 단단한 것은 좀 더 맵고 말랑한 것은 매운맛이 덜하니 기호에 따라 고르셔요^^.새송이버섯은 가로로 2등분 한 뒤 0.3cm 두께로 슬라이스해주세요.쫄깃한 식감의 새송이버섯은 꼭 넣어주세요.계란보다 더 맛 좋은 별미랍니다.슬라이스하는 것이 번거롭다면 미니새송이버섯을 사

## **1 Fast Text 모델링**
https://pypi.org/project/fasttext/
1. Gensim 보다 빠르다더라...
1. https://github.com/facebookresearch/fastText
1. **[Wiki 백과로 미리 훈련한 사전을 제공](https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.ko.align.vec)**
1. [실습 예제자료를 다운 루는 블로그](https://inspiringpeople.github.io/data%20analysis/word_embedding/)

In [51]:
text_data = """반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.본문 기타 기능.조림 밑반찬으로 대표적인 장조림.
지방이 적고 단백질은 풍부한.을 이용해 만들어봤어요.고기를 잘게 찢어 넣어 식감은 더 부드럽게.과 제철을 맞이한.
까지 넉넉하게 넣어 만들었어요.꽈리고추는 일반 고추에 비해 쭈글쭈글하니 못생겼지만.
비타민A와 C 등의 영양가가 풍부하고 매운맛이 덜해 조림이나 찜으로 먹기에 좋아요^^.닭가슴살 장조림 재료.
조리시간 1시간 이내 난이도 중.생 닭가슴살 200g 물 1L 5컵 청주 2큰술 마늘 4쪽 생강 마늘 크기 1톨.
새송이버섯 2개 꽈리고추 200g 약 3줌.양념 고기 육수 300ml 1.5컵 양조간장 100ml 0.5컵 꿀 2큰술 표고버섯가루 1 2큰술
후춧가루 약간.괄호 속 기재된 컵 은 종이컵 기준입니다.꿀 대신 올리고당으로 대체 가능합니다.고기 삶은 물은 양념에 사용할 거예요."""

In [52]:
import fasttext
script_file = './w2v_script.txt'
# model = fasttext.skipgram(script_file, 'model', lr=0.1, dim=300)
# model = fasttext.skipgram('train.txt', 'model', lr=0.1, dim=300)
# model = fasttext.skipgram(text_data, 'model', lr=0.1, dim=300)
model = fasttext.train_unsupervised(script_file, model='skipgram', dim=300, lr=0.1)

ValueError: ./w2v_script.txt cannot be opened for training!

In [53]:
import fasttext

# Skipgram model :
model = fasttext.train_unsupervised('data.txt', model='skipgram')

# or, cbow model :
model = fasttext.train_unsupervised('data.txt', model='cbow')

ValueError: data.txt cannot be opened for training!

In [54]:
print(model.words)   # list of words in dictionary
print(model['king']) # get the vector of the word 'king'

NameError: name 'model' is not defined

In [1]:
# 학습결과 Model 로 저장하기
model.save_model("model_filename.bin")

In [ ]:
# 저장된 Model 객체 불러오기
model = fasttext.load_model("model_filename.bin")

In [ ]:
import fasttext
model = fasttext.train_supervised('data.train.txt')
print(model.words)
print(model.labels)

In [ ]:
model.words         # equivalent to model.get_words()
model.labels        # equivalent to model.get_labels()

In [ ]:
model['king']       # equivalent to model.get_word_vector('king')
'king' in model     # equivalent to `'king' in model.get_words()`

In [ ]:
# 개별 Token 의 가중치 확인하기
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test.txt'))

In [ ]:
# 학습 모델을 활용하여 판단하기
model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
# 출력할 결과값을 사용자가 지정 가능 (k=3)
model.predict("Which baking dish is best to bake a banana bread ?", k=3)

In [ ]:
# If you want to predict more than one sentence you can pass an array of strings 
model.predict(["Which baking dish is best to bake a banana bread ?", "Why not put knives in the dishwasher?"], k=3)

In [ ]:
# with the previously trained `model` object, call :
model.quantize(input='data.train.txt', retrain=True)

# then display results and save the new model :
print_results(*model.test(valid_data))
model.save_model("model_filename.ftz")

## **1 Naver Link 목록**
OpenAPI 로 수집한 결과값 저장된 목록 List 원본의 수정보완
1. Naver OPEN API 결과 **Token 이 유효값인지** 확인 가능했다 (Oh!!)
1. 네이버 블로그 목록만 정리한 DataBase 만들기
1. 목록을 4개로 나워서 2개의 서버에서 돌리며 저장하기

In [3]:
import pandas as pd
df = pd.read_csv('data/naver_posts_sample.csv')
df.head(3)

,Query,Posts
0,가슴살_계란_닭_장조림,반찬 볶음 조림.부드러운 닭가슴살 장조림 만들기.베리츄..2017.9.1.22 32...
1,간장_불고기_제육_파채,오늘의 식탁.[반조리 밥아저씨] 12월 12일 식탁 버섯제육간장불고기 파채 감자고추...
2,감자_국_수제비,나의이야기.아침에 먹고 남은 감자국으로 수제비 만드는법 하기 위해 수제비 반죽만들기...


In [14]:
querys = sorted(list(set("_".join(df.Query.tolist()).split("_"))))
print(len(querys)), " ".join(querys)

86


(None,
 '가슴살 간장 감자 강정 겉절이 계란 계란말이 고추 고추장 골뱅이 국 굴 김치 깨 꼬치 꽃게 너비아니 달래 닭 데리야끼 도라지생채 돈 돈육 된장국 두부 들깨 떡국 만두 매콤 무 무침 물 미나리 미소 미역 버섯 볶음 부추 불고기 브로콜리 사골 사태 상추 새송이 새우젓 샐러드 소고기 소스 수제비 순두부 시금치나물 야채 양상추 양파 어묵 엿 오이 오징어 우동 유자 장국 장조림 장포 적 제육 조림 짬뽕 찜 찹쌀 채 채소 청경채 초무침 치어 치즈 칠리 콩나물 탕 탕수 파채 팽이 해물탕 햄 황도 후랑크 후르츠')

## **1 Naver Link 목록**
OpenAPI 로 수집한 결과값 저장된 목록 List 원본의 수정보완
1. Naver OPEN API 결과 **Token 이 유효값인지** 확인 가능했다 (Oh!!)
1. 네이버 블로그 목록만 정리한 DataBase 만들기
1. 목록을 4개로 나워서 2개의 서버에서 돌리며 저장하기